In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

In [24]:
from time_series import revert

In [25]:
qb2 = pd.read_csv('../data/dashboard/qb/qb_smoothed.csv')
rb2 = pd.read_csv('../data/dashboard/rb/rb_smoothed.csv')
wr2 = pd.read_csv('../data/dashboard/wr/wr_smoothed.csv')

qb2.rename(columns={'player_display_name': 'player'}, inplace=True)
rb2.rename(columns={'player_display_name': 'player'}, inplace=True)
wr2.rename(columns={'player_display_name': 'player'}, inplace=True)

qb2['position'] = 'QB'
rb2['position'] = 'RB'
wr2['position'] = 'WR'

In [26]:
qb2 = qb2.rename(columns={'predicted_smoothed_salary': 'predicted_salary'})
rb2 = rb2.rename(columns={'predicted_smoothed_salary': 'predicted_salary'})
wr2 = wr2.rename(columns={'predicted_smoothed_salary': 'predicted_salary'})

In [27]:
reverting = revert(qb2, position = 'QB', year = 2024)
qb = reverting.unnormalize('smooth')

reverting = revert(rb2, position = 'RB', year = 2024)
rb = reverting.unnormalize('smooth')

reverting = revert(wr2, position = 'WR', year = 2024)
wr = reverting.unnormalize('smooth')

In [28]:
qb.sample(2)

,Unnamed: 0,player,year_signed,games,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_fumbles,passing_first_downs,passing_epa,passing_2pt_conversions,pacr,dakota,rushing_yards,rushing_tds,fantasy_points,salary_per_year_x,mean_adj_salary_per_year,med_adj_salary_per_year,salary_per_year_y,smoothed_salary_per_year,predicted_median_salary,predicted_mean_salary,predicted_salary,position,SAD,smoothed_apy,reverted_pred_salary
6,9,Russell Wilson,2024.0,15.0,294.5,465.0,3297.0,21.0,9.5,50.0,5.5,138.5,-36.166604,1.0,0.894569,0.067798,309.0,3.0,240.83,1.21,-0.252200,0.475908,1.21,-0.393551,38.159949,2.822775,5.270568,QB,3.952081,2.765347,23.595057
12,25,Jameis Winston,2024.0,6.0,25.0,47.0,264.0,2.0,3.0,2.0,0.0,10.0,-9.789020,0.0,0.432787,-0.020195,-6.0,0.0,11.96,4.00,0.144765,5.494295,4.00,0.312406,2.344567,-0.122642,-0.345981,QB,3.952081,2.765347,1.398003


In [29]:
qb_best_features = ['player', 'position','passing_tds', 'passing_epa','rushing_yards', 'salary_per_year_y', 'reverted_pred_salary']
rb_best_features = ['player', 'position', 'receiving_yards_after_catch', 'rushing_yards','rushing_first_downs','salary_per_year','reverted_pred_salary']
wr_best_features = ['player', 'position', 'receiving_yards', 'target_share', 'receiving_first_downs', 'salary_per_year', 'reverted_pred_salary']

qb = qb[qb_best_features]
rb = rb[rb_best_features]
wr= wr[wr_best_features]

rename_cols = lambda df: df.rename(columns={df.columns[2]: 'feature_1', df.columns[3]: 'feature_2', df.columns[4]: 'feature_3', 'player':'name', 'salary_per_year_y': 'actual_salary', 'reverted_pred_salary': 'predicted_salary', 'salary_per_year': 'actual_salary'})
qb = rename_cols(qb)
rb = rename_cols(rb)
wr = rename_cols(wr)

full  = pd.concat([qb, rb, wr], ignore_index=True)
full.sample(5)

,name,position,feature_1,feature_2,feature_3,actual_salary,predicted_salary
93,Salvon Ahmed,RB,109.0,61.000000,3.0,1.292500,0.881100
83,Tony Pollard,RB,354.0,1061.000000,60.0,7.250000,6.968463
151,David Bell,WR,221.0,0.084932,11.0,1.253285,2.349700
226,Andy Isabella,WR,0.0,0.033333,0.0,1.133000,0.757590
97,Ke'Shawn Vaughn,RB,1.0,42.000000,0.0,1.125000,0.893725


In [30]:
numerics = ['feature_1', 'feature_2', 'feature_3', 'actual_salary', 'predicted_salary']
full[numerics] = full[numerics].round(2)
full = full[full['name'] != 'Russell Wilson']
full = full[full['name'] != 'Tee Higgins']
print(full.shape)

## undervalued 
full[(full.predicted_salary - full.actual_salary) > 3]

(226, 7)


,name,position,feature_1,feature_2,feature_3,actual_salary,predicted_salary
9,Desmond Ridder,QB,7.0,-14.43,128.5,0.98,6.35
22,Joshua Dobbs,QB,13.0,-80.75,421.0,2.25,10.33
26,Baker Mayfield,QB,34.0,67.00,194.0,33.33,43.27
29,Jake Browning,QB,12.0,22.30,127.0,0.97,16.03
62,Raheem Mostert,RB,216.0,1045.00,61.0,4.13,8.21
107,Tim Patrick,WR,734.0,0.17,39.0,1.30,9.47
147,Rashid Shaheed,WR,719.0,0.14,27.0,3.09,8.11
192,Mike Evans,WR,1450.0,0.25,61.0,20.50,28.22
207,Keelan Doss,WR,33.0,0.16,1.0,1.06,4.07
224,Josh Reynolds,WR,740.0,0.12,39.0,4.50,11.06


In [31]:
## overvalued 
full[(full.actual_salary - full.predicted_salary) > 10]

,name,position,feature_1,feature_2,feature_3,actual_salary,predicted_salary
0,Jared Goff,QB,31.5,104.48,48.5,53.0,41.61
1,Tua Tagovailoa,QB,27.5,103.50,84.5,53.1,36.62
2,Dak Prescott,QB,33.5,87.49,257.5,60.0,41.26
7,Kirk Cousins,QB,24.5,33.91,61.5,45.0,31.40
117,Darnell Mooney,WR,414.0,0.15,20.0,13.0,2.57


In [32]:
### joining in height, weight, and age

players = pd.read_csv('../data/raw/players.csv')

full = full.merge(
    players[['display_name', 'birth_date', 'height', 'weight']], 
    how='left', 
    left_on='name', 
    right_on='display_name'
)


full = full.drop_duplicates(subset='name', keep='first')
full['birth_date'] = pd.to_datetime(full['birth_date'], errors='coerce')
today = pd.Timestamp.today()
full['age'] = (today - full['birth_date']).dt.days // 365

full = full.drop(columns=['display_name'])
full = full.drop(columns=['birth_date'])

full = full[['name', 'height', 'weight', 'age', 'position', 'feature_1', 'feature_2', 'feature_3', 'actual_salary', 'predicted_salary']]
print(full.shape)
full.sample(5)

(226, 10)


,name,height,weight,age,position,feature_1,feature_2,feature_3,actual_salary,predicted_salary
202,DeAndre Carter,68.0,190.0,32.0,WR,39.0,0.03,0.0,1.20,1.00
74,Jeff Wilson,NaN,NaN,NaN,RB,78.0,188.00,8.0,1.75,1.09
87,Ty Johnson,70.0,210.0,27.0,RB,71.0,198.00,14.0,1.29,1.11
234,Cedrick Wilson Jr.,NaN,NaN,NaN,WR,333.0,0.11,16.0,2.88,1.63
244,Andy Isabella,69.0,190.0,28.0,WR,0.0,0.03,0.0,1.13,0.76


In [39]:
full.sample(5)

,name,height,weight,age,position,feature_1,feature_2,feature_3,actual_salary,predicted_salary
35,Bailey Zappe,73.0,220.0,26.0,QB,6.0,-81.80,83.0,0.98,2.08
226,D.J. Montgomery,73.0,201.0,28.0,WR,56.0,0.13,3.0,0.22,1.78
104,Ke'Shawn Vaughn,70.0,215.0,27.0,RB,1.0,42.00,0.0,1.12,0.89
119,Danny Gray,74.0,199.0,26.0,WR,10.0,0.06,1.0,0.22,0.80
145,Marquise Brown,69.0,170.0,27.0,WR,574.0,0.25,32.0,7.00,5.59


In [40]:
full.to_csv('../data/dashboard/full.csv')

In [70]:
## QB

# feature 1 = fantasy points | feature 2 = passing tds | feature 3 = rushing yards

## RB

# feature 1 = fantasy points | feature 2 = rushing yards | feature 3 = target share

## WR

# feature 1 = fantasy points | feature 2 = target share | feature 3 = receiving yards